In [1]:
import ee
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import geemap
import os
import time
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import geopandas as gpd
import pandas as pd
import numpy as np
from mpi4py import MPI

In [2]:
##########################################################################################
# Set Parameters
##########################################################################################

# area of interest params
# choose bounding area format ('STATE', 'COUNTRY', 'BBOX', 'HUC', 'SHP'):
ROI = 'SHP'

# if ROI = BBOX or SHP (path to .geojson or .shp, otherwise ''):
IN_PATH = '/mnt/poseidon/remotesensing/arctic/data/vectors/supplementary/tundra_alaska/tundra_alaska.shp'
# if ROI = STATE or COUNTRY (administrative boundaries, otherwise None):
COUNTRY = None
# if ROI = HUC, state abbreviation for HUC, if STATE, fulls state name:
STATE = None # 'AK' 
# if ROI = HUC (list of HUC6 units):
HUCLIST = None # must be list: ['190604', '190603', '190602']

##########################################################################################

# buffer around point to find median of intersecting pixel values
POINTBUFFER = 30 # meters

##########################################################################################

# output file
DIR_PATH = '/mnt/poseidon/remotesensing/arctic/data/rasters/S1GRD/training_test04'
try:
    if os.path.isdir(DIR_PATH) == False:
        os.mkdir(DIR_PATH)
except Exception as e:
    pass

##########################################################################################

# data Information
IDCOL = 'Site Code'
SCALE = 10
BANDS = ['elevation']
start_date = date(2019, 6, 1)# Y-M-D (2019, 1, 1)
end_date = date(2019, 8, 31) # Y-M-D minus 5 for even 'days' intervals (6 days for 2020)
TIMESTEP = None # 'months', 'days', or None
DAYS = '' # if TIMESTEP = days
MONTHS = '' # if TIMESTEP = 'months': years * 12

In [3]:
##########################################################################################
# Create ee_to_df function for exporting
##########################################################################################

def ee_to_df(ee_object, col_names, sort_columns=False):
    if isinstance(ee_object, ee.Feature):
        ee_object = ee.FeatureCollection([ee_object])

    if not isinstance(ee_object, ee.FeatureCollection):
        raise TypeError("ee_object must be an ee.FeatureCollection")

    try:
        property_names = ee_object.first().propertyNames().sort().getInfo()
        #data = ee_object.map(lambda f: ee.Feature(None, f.toDictionary(property_names)))
        data = ee_object
        data = [x["properties"] for x in data.getInfo()["features"]]
        df = pd.DataFrame(data)

        if col_names is None:
            col_names = property_names
            col_names.remove("system:index")
        elif not isinstance(col_names, list):
            raise TypeError("col_names must be a list")

        df = df[col_names]

        if sort_columns:
            df = df.reindex(sorted(df.columns), axis=1)

        return df
    
    except Exception as e:
        raise Exception(e)

In [4]:
rank = 1
size = 4

In [5]:
##########################################################################################
# Set GEE Vector Bounds
##########################################################################################

# Import admin data and select country to create grid around
if ROI == 'STATE':
    grid_location_ee = (ee.FeatureCollection("FAO/GAUL/2015/level1")
                        .filterMetadata('ADM0_NAME', 'equals', COUNTRY)
                        .filterMetadata('ADM1_NAME', 'equals', STATE))

elif ROI == 'COUNTRY':
    grid_location_ee = (ee.FeatureCollection("FAO/GAUL/2015/level1")
                        .filterMetadata('ADM0_NAME', 'equals', COUNTRY))
	
elif ROI == 'BBOX':
	grid_location_ee = geemap.geojson_to_ee(IN_PATH)
    
elif ROI == 'HUC':
    grid_location_ee = (ee.FeatureCollection("USGS/WBD/2017/HUC06")
                        .filter(ee.Filter.inList('huc6', HUCLIST)))
    
elif ROI == 'SHP':
    geodataframe = gpd.read_file(IN_PATH)
    grid_location_ee = geemap.geopandas_to_ee(geodataframe)
    
else:
    print('Invalid region of interest. Check STATE, COUNTRY, HUC')

In [6]:
##########################################################################################
# Get Sampling Points
##########################################################################################

##########################################################################################
# AKVEG test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'VEG_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
akv_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(akv_geom))
akv_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# ABR_RS test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'ABR_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
abr_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(abr_geom))
abr_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# AKAVA test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'AVA_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
ava_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(ava_geom))
ava_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# NEON
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'NEO_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
neo_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(neo_geom))
neo_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# Seward test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'SP_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
nge_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(nge_geom))
nge_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# combine
obs_geom = pd.concat([akv_geom, abr_geom, ava_geom, neo_geom, nge_geom], 
                     axis=0, 
                     ignore_index=True)
print(len(obs_geom))

# create ee object (feature collection)
obs_geom = obs_geom.reset_index()
obs_points = geemap.df_to_ee(obs_geom,
                             latitude='latitude',
                             longitude='longitude')
print(obs_points.size().getInfo())

##########################################################################################
#sub-select points and extract geometry

# select points that intercept HUC
samplepoints = obs_points.filterBounds(grid_location_ee)

# create dictionary of grid coordinates
points_dict = samplepoints.getInfo()
feats = points_dict['features']

# get ID column
unique_ids = []
for f in feats:
    id = f['properties'][IDCOL]
    unique_ids.append(id)

# Create a list of several ee.Geometry.Polygons
points = []
for f in feats:
    coords = f['geometry']['coordinates']
    point = ee.Geometry.Point(coords)
    # create buffer around point for later reduce regions
    buffered = point.buffer(POINTBUFFER)
    points.append(buffered)

# Make a feature collection for export purposes
points_ee = ee.FeatureCollection(points)
print(f'{len(points)} {POINTBUFFER}-meter buffered points.')

185
107
90
47
86
515
515
514 30-meter buffered points.


In [9]:
# get elevation, slope, aspect
dem = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic')
elevation = dem.select('elevation').clip(grid_location_ee)
terrain = ee.Terrain.products(elevation).clip(grid_location_ee)

# set vis
elevation_vis = {'min': -50.0,
                 'max': 1000.0,
                 'palette': ['0d13d8', '60e1ff', 'ffffff']}

# make map
m = geemap.Map()
m.center_object(grid_location_ee)
m.addLayer(elevation, elevation_vis, 'Arctic DEM', True)
m.addLayer(terrain.select('slope'), {'min': 0, 'max': 89.99}, 'Slope', True)
m.addLayer(terrain.select('aspect'), {'min': 0, 'max': 359.99}, 'Aspect', True)
m.addLayer(terrain.select('hillshade'), {'min': 0, 'max': 255}, 'Hillshade', True)
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…